# Automating Scripts with Docker Compose

### Introduction

In the last lesson, we focused on Dockerizing our flask application.  In this lesson, we'll see how we can use Docker compose to further automate configuration like port mapping, setting environmental variables, and bind mounts.

### Reviewing our Application

With our Flask application in the last lesson, we wrote code that returned a list of users upon a GET request to `/users`.

```python
# project/__init__.py
from flask import Flask, jsonify

app = Flask(__name__)

@app.route('/users')
def index():
    return jsonify({'users': ['bart simpson']})

if __name__ == '__main__':
    app.run(debug=True)
```

And we then embedded this in a docker image with the following `Dockerfile`.

```Dockerfile
FROM python:3.7-alpine
WORKDIR /usr/src/app
COPY ./requirements.txt .
RUN pip3 install -r requirements.txt
COPY . .
CMD ["flask", "run", "--host=0.0.0.0"]
```

Now to actually run this so that we can access our flask application, we need to first build our image:
`docker build -t flask_docker .`

And then call `docker run` like so: 

> `docker run -p 5000:5000 -e FLASK_APP=project/__init__.py flask_docker`

Now calling the run file properly involves a good amount of knowledge about the project.  We need to know/remember to map our ports, and to specify an environmental variable.  So let's use docker-compose to specify this configuration in `yml` file.

### Introducing Docker Compose

To docker-compose with the following `docker-compose.yml` file:

```python
version: '3.7'
services:
  web:
    image: flask_docker
    ports:
      - 5000:5000
    environment:
      - FLASK_APP=project/__init__.py
```

Ok, now instead of calling `docker run` directly, we can call our `docker-compose` file with the following command: 

* `docker-compose up`

<img src="./docker-compose-web-db.png" width="80%">

As we know, the `docker-compose.yml` file defines one service.  We would like postgres to be our second service.  Lucky for us, there is already a [postgres image on dockerhub](https://hub.docker.com/_/postgres) for us.  And we can modify our `docker-compose.yml` as our second service.

```python

version: '3.7'
services:
  web:
    build:
      context: ./web-app
      dockerfile: Dockerfile
    ports:
      - 3000:5000
    environment:
      - FLASK_APP=project/__init__.py
      - FLASK_ENV=development
    volumes:
      - .:/web/flask
  db:
    image: postgres:12-alpine
    ports:
      - 5435:5432
    environment:
      - POSTGRES_USER=default-user
      - POSTGRES_PASSWORD=password
```

Let's pay attention to a couple of things with our db service.  The first is that here we are not using a custom Dockerfile, but rather just using the image that we see in our Dockerhub.  
> If we wanted to have a custom postgres setup, we would just use the `build` like before and use the `postgres` image as the base image in the respective Dockerfile. 

Next, when we connect to our postgres database, we also connect through a port.  Here, we link our host's port of 5435 to the container's port of 5432 where postgres is running.  We can see this by Googling around, or by looking at the [Dockerfile](https://github.com/docker-library/postgres/blob/d6e8fe3240b3d2c5d1a03f005360710812714163/12/alpine/Dockerfile) that ends with: 

```python
EXPOSE 5432
CMD ["postgres"]
```

Finally, we set the `POSTGRES_USER=postgres` and `POSTGRES_PASSWORD=postgres` as environment variables for accessing postgres under the `environment` flag.

### Linking to the Database

With connecting to a postgres database, the format is the following, we can connect if we have the following information: 

* user
* pwd
* host
* port
* db

The user and password we have already defined as `default-user` and `password`.  The host is normally a host-url where the database is hosted, but within docker-compose, we can just use the image name we provided.  So the host is simply `db`.  Then we still need to create a database for development, and one for test and production environments.  We will name that development database `development`.  So following the pattern above, we can use some Python to format our postgres url correctly.  

In [11]:
user = 'default-user'
pwd = 'password'
port = '5432'
host = 'db'
database = 'development'

'postgres://%s:%s@%s:%s/%s' % (user, pwd, host, port, db)

'postgres://default-user:password@db:5432/development'

So if we want to connect a sql client to the development database this is how we do it.  

```python
services:
    web:
        environment:
          - FLASK_APP=project/__init__.py
          - FLASK_ENV=development
          - APP_SETTINGS=project.config.DevelopmentConfig
          - DATABASE_URL=postgres://default-user:password@db:5432/development
          - DATABASE_TEST_URL=postgres://default-user:password@db:5432/test
          - DATABASE_PROD_URL=postgres://default-user:password@db:5432/prod
```

This sets up the environment variables in our web container, which point to the postgres database.  Then, we can update the config file in our flask app to reference these variables and thus the postgres databases.

```python

# users/project/config.py
import os
class BaseConfig:
    """Base configuration"""
    TESTING = False
    SQLALCHEMY_TRACK_MODIFICATIONS = False

class DevelopmentConfig(BaseConfig):
    """Development configuration"""
    SQLALCHEMY_DATABASE_URI = os.environ.get('DATABASE_URL')  

class TestingConfig(BaseConfig):
    """Testing configuration"""
    TESTING = True
    SQLALCHEMY_DATABASE_URI = os.environ.get('DATABASE_TEST_URL') 

class ProductionConfig(BaseConfig):
    """Production configuration"""
    SQLALCHEMY_DATABASE_URI = os.environ.get('PROD_DATABASE_URL')
```

And then take a look at our `__init__.py` file below.  There, we connect have our `APP_SETTINGS` environment variable set as `DevelopmentConfig`, which referencess the respective class in the `config.py` file, which references the `DATABASE_URL`, which points to `postgres://default-user:password@db:5432/development`.  So when we are in our development environment, SQLAlchemy connects to `postgres://default-user:password@db:5432/development`.

```python
import os  # new
from flask import Flask, jsonify
from flask_restful import Resource, Api
from flask_sqlalchemy import SQLAlchemy  # new

app = Flask(__name__)
api = Api(app)

app_settings = os.getenv('APP_SETTINGS')  # new
app.config.from_object(app_settings)      # new

db = SQLAlchemy(app)
```

Now, we need a script that will create these databases.  So we add a new folder of `db` and a file of `create.sql` to write the following script.

`project/db/create.sql`
```sql
CREATE DATABASE users_prod;
CREATE DATABASE users_dev;
CREATE DATABASE users_test;
```

### Final Touches

Our environment is almost setup.  There are just a few missing items:

1. We need to persist changes to the database 
    * This sounds like the job of a volume.  We can create a volume mounted in our `db` container with the following.
    
```python
  db:
    image: postgres:12-alpine
    ports:
      - 5435:5432
    environment:
      - POSTGRES_USER=default-user
      - POSTGRES_PASSWORD=password
    volumes:
      - "dbdata:/var/lib/postgresql/data"
```

So we create a volume named `dbdata` that mounts to our postgres image at `var/lib/postgresql/data`.  How did we know to mount our volume to that location in the postgres image?  It tells us in the [postgres image documentation](https://hub.docker.com/_/postgres/).

2. We need to ensure that the postgres image runs the `create.sql` file

There are a couple of different ways that we can achieve this.  One is to simply extend the postgres image by creating a Dockerfile for this image, and running the file.

A second is to user our `docker-compose.yml` file to override the command in the used image.  Let's take this separate approach.

```python
  db:
    image: postgres:12-alpine
    ports:
      - 5435:5432
    environment:
      - POSTGRES_USER=default-user
      - POSTGRES_PASSWORD=password
    volumes:
      - "dbdata:/var/lib/postgresql/data"
    command:
        - "init.db.sh"
```

**Add init.db.sh file to this section** 

3. An order dependency between images

Finally, it matters to us which image is booted up first.  Think about it, if our flask app tries to connect to the `db` image before it is up and running, there will be problems.  We can ensure that our `db` image boots up before the `web` image, by adding a `depends_on: db` key to the `web` image.  
```
web:
    depends_on: 
      - db
```
With these changes in place our `docker-compose.yml` file looks like the following.

```python

version: '3.7'
services:
  web:
    build:
      context: ./web-app
      dockerfile: Dockerfile
    ports:
      - 3000:5000
    environment:
      - FLASK_APP=project/__init__.py
      - FLASK_ENV=development
      - APP_SETTINGS=project.config.DevelopmentConfig
      - DATABASE_URL=postgres://default-user:password@db:5432/development
      - DATABASE_TEST_URL=postgres://default-user:password@db:5432/test
      - DATABASE_PROD_URL=postgres://default-user:password@db:5432/prod
    volumes:
      - .:/web/flask
    depends_on: 
      - db
  db:
    image: postgres:12-alpine
    ports:
      - 5435:5432
    environment:
      - POSTGRES_USER=default-user
      - POSTGRES_PASSWORD=password
    volumes:
      - "dbdata:/var/lib/postgresql/data"
    command:
        - "init.db.sh"
```

* maybe need to change project.config.DevelopmentConfig to web-app.config

Now we simply have to run `docker-compose up` and we can view our web app loaded up and available on port `3000`.

### Resources

[docker with volumes](https://www.saltycrane.com/blog/2019/01/how-run-postgresql-docker-mac-local-development/)

http://www.ameyalokare.com/docker/2017/09/20/nginx-flask-postgres-docker-compose.html